# Code for calculating the final dataframes for regression of the effect of key moments on the flow of momentum. 

In [ ]:
#2 minute time windows pitch control and PCV
import os
from progressbar import ProgressBar
import pandas as pd
import pickle
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code'
pc_matches = []

#create a list with all pitch control matches
for file in os.listdir(path):
    if 'pc_over_last_minute' in file:
        pc_matches.append(file[19:26])

#create empty dataframe
final_df = pd.DataFrame()
pbar = ProgressBar()

#iterate over all matches
for game in pbar(pc_matches):
    
    #import dataframe with pitch control and PCV 
    infile = open('pc_over_last_minute' + game + '.p','rb')
    df_pc = pickle.load(infile)
    infile.close()
    
    #create a dataframe for the first and second half
    fh = df_pc.loc[df_pc['Half'] == 1]
    sh = df_pc.loc[df_pc['Half'] == 2]
    
    #import the shots and goals dataframe
    infile = open('shots and goals event dataframes.p','rb')
    df_shots = pickle.load(infile)
    infile.close()
    
    #filter the shots and goals dataframe on the current match
    df_shots = df_shots.loc[df_shots['match_id'] == game]
    
    #for each shot and goals: add whether it is favorable or unfavorable for the home team
    type_key = []
    key_mom = []
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['team'] == 'Home':
            key_mom.append('favorable')
            type_key.append(1)
        else:
            key_mom.append('unfavorable')
            type_key.append(-1)

    df_shots['key_moment'] = key_mom
    df_shots['type_key'] = type_key

    df_shots = df_shots[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'outcome', 'match_id', 'key_moment', 'type_key']]
    
    
    mean_pc_b_home = []
    mean_wpc_b_home = []
    mean_pc_a_home = []
    mean_wpc_a_home = []
    mean_pc_b_away = []
    mean_wpc_b_away = []
    mean_pc_a_away = []
    mean_wpc_a_away = []
    index_min_below = []
    index_min_above = []
    
    #create seperate dataframes for the shots and goals for the first and second half
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    
    #for each shot or gaol in the first half: add the home and away pitch control and PCV values for the time windows
    for i in range(len(df_shots_fh)):
        minute = float(df_shots_fh.iloc[i]['minute'])
        second = float(df_shots_fh.iloc[i]['seconds'])
        index_above = fh[fh['Time'] > float(minute + second/60)].iloc[0].name
        try:
            index_below = fh[fh['Time'] <= float(minute + second/60)].iloc[-1].name
        except:
            index_below = index_above
        
        if int(df_shots_fh.iloc[i]['minute']) < 2:
            #if minute smaller than one: only take the time window in the first mnute
            if int(df_shots_fh.iloc[i]['minute']) < 1:
                index_2min_above = fh.index[fh.index <= index_above +2*60*25].max()
                index_2min_below = fh.index[fh.index >= index_below -2*60*25].min()
                index_1min_below = fh.index[fh.index >= index_below -60*25].min()
                mean_pc_b_home.append(fh.loc[index_below]['PC home previous minute'])
                mean_wpc_b_home.append(fh.loc[index_below]['Weighted PC home previous minute'])
                mean_pc_b_away.append(fh.loc[index_below]['PC away previous minute'])
                mean_wpc_b_away.append(fh.loc[index_below]['Weighted PC away previous minute'])
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)
            #if minute is smaller than two and bigger than one: weight the one minute time windows as one is bigger than the other.
            else:
                time = second/60
                index_2min_above = fh.index[fh.index <= index_above +2*60*25].max()
                index_2min_below = fh.index[fh.index >= index_below -2*60*25].min()
                index_1min_below = fh.index[fh.index >= index_below -60*25].min()
                mean_pc_b_home.append((fh.loc[index_below]['PC home previous minute'] + (time) * fh.loc[index_1min_below]['PC home previous minute'])/(1+time))
                mean_wpc_b_home.append((fh.loc[index_below]['Weighted PC home previous minute'] + (time) * fh.loc[index_1min_below]['Weighted PC home previous minute'])/(1+time))
                mean_pc_b_away.append((fh.loc[index_below]['PC away previous minute'] + (time) * fh.loc[index_1min_below]['PC away previous minute'])/(1+time))
                mean_wpc_b_away.append((fh.loc[index_below]['Weighted PC away previous minute'] + (time) * fh.loc[index_1min_below]['Weighted PC away previous minute'])/(1+time))
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)
            
            mean_pc_a_home.append((fh.loc[index_2min_above]['PC home previous minute'] + fh.loc[index_2min_above-60*25]['PC home previous minute'])/2)
            mean_wpc_a_home.append((fh.loc[index_2min_above]['Weighted PC home previous minute'] + fh.loc[index_2min_above-60*25]['Weighted PC home previous minute'])/2)
            mean_pc_a_away.append((fh.loc[index_2min_above]['PC away previous minute'] + fh.loc[index_2min_above-60*25]['PC away previous minute'])/2)
            mean_wpc_a_away.append((fh.loc[index_2min_above]['Weighted PC away previous minute'] + fh.loc[index_2min_above-60*25]['Weighted PC away previous minute'])/2)
        
        #same at the end of the match
        elif fh.iloc[-1]['Time'] - float(minute + second/60) < 2:
            if fh.iloc[-1]['Time'] - float(minute + second/60) < 1:
                index_2min_above = fh.index[fh.index <= index_above +2*60*25].max()
                index_2min_below = fh.index[fh.index >= index_below -2*60*25].min()
                index_1min_below = fh.index[fh.index >= index_below -60*25].min()
                index_1min_above = fh.index[fh.index <= index_above +60*25].max()
                mean_pc_a_home.append(fh.loc[index_1min_above]['PC home previous minute'])
                mean_wpc_a_home.append(fh.loc[index_1min_above]['Weighted PC home previous minute'])
                mean_pc_a_away.append(fh.loc[index_1min_above]['PC away previous minute'])
                mean_wpc_a_away.append(fh.loc[index_1min_above]['Weighted PC away previous minute'])
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)

            else:
                time = second/60
                index_2min_above = fh.index[fh.index <= index_above +2*60*25].max()
                index_2min_below = fh.index[fh.index >= index_below -2*60*25].min()
                index_1min_below = fh.index[fh.index >= index_below -60*25].min()
                index_1min_above = fh.index[fh.index <= index_above +60*25].max()
                mean_pc_a_home.append(((1-time)*fh.loc[index_2min_above]['PC home previous minute'] + fh.loc[index_1min_above]['PC home previous minute'])/(2-time))
                mean_wpc_a_home.append(((1-time)*fh.loc[index_2min_above]['Weighted PC home previous minute'] + fh.loc[index_1min_above]['Weighted PC home previous minute'])/(2-time))
                mean_pc_a_away.append(((1-time)*fh.loc[index_2min_above]['PC away previous minute'] + fh.loc[index_1min_above]['PC away previous minute'])/(2-time))
                mean_wpc_a_away.append(((1-time)*fh.loc[index_2min_above]['Weighted PC away previous minute'] + fh.loc[index_1min_above]['Weighted PC away previous minute'])/(2-time))
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)

            mean_pc_b_home.append((fh.loc[index_below]['PC home previous minute'] + fh.loc[index_1min_below]['PC home previous minute'])/2)
            mean_wpc_b_home.append((fh.loc[index_below]['Weighted PC home previous minute'] + fh.loc[index_1min_below]['Weighted PC home previous minute'])/2)
            mean_pc_b_away.append((fh.loc[index_below]['PC away previous minute'] + fh.loc[index_1min_below]['PC away previous minute'])/2)
            mean_wpc_b_away.append((fh.loc[index_below]['Weighted PC away previous minute'] + fh.loc[index_1min_below]['Weighted PC away previous minute'])/2)

        #normal situations where we have full two-minute windows
        else:
            index_2min_above = fh.index[fh.index <= index_above +2*60*25].max()
            index_2min_below = fh.index[fh.index >= index_below -2*60*25].min()
            index_1min_below = fh.index[fh.index >= index_below -60*25].min()
            mean_pc_b_home.append((fh.loc[index_below]['PC home previous minute'] + fh.loc[index_1min_below]['PC home previous minute'])/2)
            mean_wpc_b_home.append((fh.loc[index_below]['Weighted PC home previous minute'] + fh.loc[index_1min_below]['Weighted PC home previous minute'])/2)
            mean_pc_a_home.append((fh.loc[index_2min_above]['PC home previous minute'] + fh.loc[index_2min_above-60*25]['PC home previous minute'])/2)
            mean_wpc_a_home.append((fh.loc[index_2min_above]['Weighted PC home previous minute'] + fh.loc[index_2min_above-60*25]['Weighted PC home previous minute'])/2)
            mean_pc_b_away.append((fh.loc[index_below]['PC away previous minute'] + fh.loc[index_1min_below]['PC away previous minute'])/2)
            mean_wpc_b_away.append((fh.loc[index_below]['Weighted PC away previous minute'] + fh.loc[index_1min_below]['Weighted PC away previous minute'])/2)
            mean_pc_a_away.append((fh.loc[index_2min_above]['PC away previous minute'] + fh.loc[index_2min_above-60*25]['PC away previous minute'])/2)
            mean_wpc_a_away.append((fh.loc[index_2min_above]['Weighted PC away previous minute'] + fh.loc[index_2min_above-60*25]['Weighted PC away previous minute'])/2)
            index_min_below.append(index_2min_below)
            index_min_above.append(index_2min_above)
            

    #do the same for the second half
    for i in range(len(df_shots_sh)):
        minute = float(df_shots_sh.iloc[i]['minute'])
        second = float(df_shots_sh.iloc[i]['seconds'])
        index_above = sh[sh['Time'] > float(minute + second/60)].iloc[0].name
        try:
            index_below = sh[sh['Time'] <= float(minute + second/60)].iloc[-1].name
        except:
            index_below = index_above
        
        if sh.iloc[-1]['Time'] - float(minute + second/60) < 2:
            if sh.iloc[-1]['Time'] - float(minute + second/60) < 1:
                index_2min_above = sh.index[sh.index <= index_above +2*60*25].max()
                index_2min_below = sh.index[sh.index >= index_below -2*60*25].min()
                index_1min_below = sh.index[sh.index >= index_below -60*25].min()
                index_1min_above = sh.index[sh.index <= index_above +60*25].max()
                mean_pc_a_home.append(sh.loc[index_1min_above]['PC home previous minute'])
                mean_wpc_a_home.append(sh.loc[index_1min_above]['Weighted PC home previous minute'])
                mean_pc_a_away.append(sh.loc[index_1min_above]['PC away previous minute'])
                mean_wpc_a_away.append(sh.loc[index_1min_above]['Weighted PC away previous minute'])
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)
            else:
                time = second/60
                index_2min_above = sh.index[sh.index <= index_above +2*60*25].max()
                index_2min_below = sh.index[sh.index >= index_below -2*60*25].min()
                index_1min_below = sh.index[sh.index >= index_below -60*25].min()
                index_1min_above = sh.index[sh.index <= index_above +60*25].max()
                mean_pc_a_home.append(((1-time)*sh.loc[index_2min_above]['PC home previous minute'] + sh.loc[index_1min_above]['PC home previous minute'])/(2-time))
                mean_wpc_a_home.append(((1-time)*sh.loc[index_2min_above]['Weighted PC home previous minute'] + sh.loc[index_1min_above]['Weighted PC home previous minute'])/(2-time))
                mean_pc_a_away.append(((1-time)*sh.loc[index_2min_above]['PC away previous minute'] + sh.loc[index_1min_above]['PC away previous minute'])/(2-time))
                mean_wpc_a_away.append(((1-time)*sh.loc[index_2min_above]['Weighted PC away previous minute'] + sh.loc[index_1min_above]['Weighted PC away previous minute'])/(2-time))
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)
                
            mean_pc_b_home.append((sh.loc[index_below]['PC home previous minute'] + sh.loc[index_1min_below]['PC home previous minute'])/2)
            mean_wpc_b_home.append((sh.loc[index_below]['Weighted PC home previous minute'] + sh.loc[index_1min_below]['Weighted PC home previous minute'])/2)
            mean_pc_b_away.append((sh.loc[index_below]['PC away previous minute'] + sh.loc[index_1min_below]['PC away previous minute'])/2)
            mean_wpc_b_away.append((sh.loc[index_below]['Weighted PC away previous minute'] + sh.loc[index_1min_below]['Weighted PC away previous minute'])/2)
        elif int(df_shots_sh.iloc[i]['minute']) < 47:
            if int(df_shots_sh.iloc[i]['minute']) < 46:
                index_2min_above = sh.index[sh.index <= index_above +2*60*25].max()
                index_2min_below = sh.index[sh.index >= index_below -2*60*25].min()
                index_1min_below = sh.index[sh.index >= index_below -60*25].min()
                mean_pc_b_home.append(sh.loc[index_below]['PC home previous minute'])
                mean_wpc_b_home.append(sh.loc[index_below]['Weighted PC home previous minute'])
                mean_pc_b_away.append(sh.loc[index_below]['PC away previous minute'])
                mean_wpc_b_away.append(sh.loc[index_below]['Weighted PC away previous minute'])
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)
            else:
                time = second/60
                index_2min_above = sh.index[sh.index <= index_above +2*60*25].max()
                index_2min_below = sh.index[sh.index >= index_below -2*60*25].min()
                index_1min_below = sh.index[sh.index >= index_below -60*25].min()
                mean_pc_b_home.append((sh.loc[index_below]['PC home previous minute'] + (time) * sh.loc[index_1min_below]['PC home previous minute'])/(1+time))
                mean_wpc_b_home.append((sh.loc[index_below]['Weighted PC home previous minute'] + (time) * sh.loc[index_1min_below]['Weighted PC home previous minute'])/(1+time))
                mean_pc_b_away.append((sh.loc[index_below]['PC away previous minute'] + (time) * sh.loc[index_1min_below]['PC away previous minute'])/(1+time))
                mean_wpc_b_away.append((sh.loc[index_below]['Weighted PC away previous minute'] + (time) * sh.loc[index_1min_below]['Weighted PC away previous minute'])/(1+time))
                index_min_below.append(index_2min_below)
                index_min_above.append(index_2min_above)
            
            mean_pc_a_home.append((sh.loc[index_2min_above]['PC home previous minute'] + sh.loc[index_2min_above-60*25]['PC home previous minute'])/2)
            mean_wpc_a_home.append((sh.loc[index_2min_above]['Weighted PC home previous minute'] + sh.loc[index_2min_above-60*25]['Weighted PC home previous minute'])/2)
            mean_pc_a_away.append((sh.loc[index_2min_above]['PC away previous minute'] + sh.loc[index_2min_above-60*25]['PC away previous minute'])/2)
            mean_wpc_a_away.append((sh.loc[index_2min_above]['Weighted PC away previous minute'] + sh.loc[index_2min_above-60*25]['Weighted PC away previous minute'])/2)

        
        else:        
            index_2min_above = sh.index[sh.index <= index_above +2*60*25].max()
            index_2min_below = sh.index[sh.index >= index_below -2*60*25].min()
            index_1min_below = sh.index[sh.index >= index_below -60*25].min()
            mean_pc_b_home.append((sh.loc[index_below]['PC home previous minute'] + sh.loc[index_1min_below]['PC home previous minute'])/2)
            mean_wpc_b_home.append((sh.loc[index_below]['Weighted PC home previous minute'] + sh.loc[index_1min_below]['Weighted PC home previous minute'])/2)
            mean_pc_a_home.append((sh.loc[index_2min_above]['PC home previous minute'] + sh.loc[index_2min_above-60*25]['PC home previous minute'])/2)
            mean_wpc_a_home.append((sh.loc[index_2min_above]['Weighted PC home previous minute'] + sh.loc[index_2min_above-60*25]['Weighted PC home previous minute'])/2)
            mean_pc_b_away.append((sh.loc[index_below]['PC away previous minute'] + sh.loc[index_1min_below]['PC away previous minute'])/2)
            mean_wpc_b_away.append((sh.loc[index_below]['Weighted PC away previous minute'] + sh.loc[index_1min_below]['Weighted PC away previous minute'])/2)
            mean_pc_a_away.append((sh.loc[index_2min_above]['PC away previous minute'] + sh.loc[index_2min_above-60*25]['PC away previous minute'])/2)
            mean_wpc_a_away.append((sh.loc[index_2min_above]['Weighted PC away previous minute'] + sh.loc[index_2min_above-60*25]['Weighted PC away previous minute'])/2)
            index_min_below.append(index_2min_below)
            index_min_above.append(index_2min_above)



    #add the pitch control lists to the shots and goals dataframe
    df_shots['PC home 2min before'] = mean_pc_b_home
    df_shots['WPC home 2min before'] = mean_wpc_b_home
    df_shots['PC home 2min after'] = mean_pc_a_home
    df_shots['WPC home 2min after'] = mean_wpc_a_home
    df_shots['PC away 2min before'] = mean_pc_b_away
    df_shots['WPC away 2min before'] = mean_wpc_b_away
    df_shots['PC away 2min after'] = mean_pc_a_away
    df_shots['WPC away 2min after'] = mean_wpc_a_away
    df_shots['index 2min before'] = index_min_below
    df_shots['index 2min after'] = index_min_above
    
    minutes_first_half = []
    minutes_second_half = []
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    #check if there are timewindows in the first half where no shots or goals took place using pure playing time. If yes, add the minutes for these time windows to a list
    for i in range(len(df_shots_fh)):
        try:
            if i == 0:
                if df_shots_fh.iloc[i]['index 2min before'] - fh.iloc[0].name > 8*60*25:
                    dif = df_shots_fh.iloc[i]['index 2min before'] - fh.iloc[0].name
                    if dif < 12*60*25:
                        minutes_first_half.append(fh.loc[fh.iloc[0].name + int(dif/2)]['Time'])
                    elif dif >= 12*60*25 and dif <16*60*25:
                        minutes_first_half.append(fh.loc[fh.iloc[0].name + int(dif/3)]['Time'])
                        minutes_first_half.append(fh.loc[fh.iloc[0].name + int((dif/3)*2)]['Time'])
                    elif dif >= 16*60*25:
                        minutes_first_half.append(fh.loc[fh.iloc[0].name + int(dif/4)]['Time'])
                        minutes_first_half.append(fh.loc[fh.iloc[0].name + int((dif/4)*2)]['Time'])
                        minutes_first_half.append(fh.loc[fh.iloc[0].name + int((dif/4)*3)]['Time'])

            if df_shots_fh.iloc[i+1]['index 2min before'] - df_shots_fh.iloc[i]['index 2min after'] > 8*60*25:
                dif = df_shots_fh.iloc[i+1]['index 2min before'] - df_shots_fh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int(dif/2)]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int(dif/3)]['Time'])
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int((dif/3)*2)]['Time'])
                elif dif >= 16*60*25:
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int(dif/4)]['Time'])
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*2)]['Time'])
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*3)]['Time'])

        except:
            if fh.iloc[-1].name - df_shots_fh.iloc[i]['index 2min after'] > 8*60*25:
                dif = fh.iloc[-1].name - df_shots_fh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int(dif/2)]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int(dif/3)]['Time'])
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int((dif/3)*2)]['Time'])
                elif dif >= 16*60*25:
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int(dif/4)]['Time'])
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*2)]['Time'])
                    minutes_first_half.append(fh.loc[df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*3)]['Time'])

    #do the same for the second half
    for i in range(len(df_shots_sh)):
        try: 
            if i == 0:
                if df_shots_sh.iloc[i]['index 2min before'] - sh.iloc[0].name > 8*60*25:
                    dif = df_shots_sh.iloc[i]['index 2min before'] - sh.iloc[0].name
                    if dif < 12*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/2)]['Time'])
                    elif dif >= 12*60*25 and dif <16*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/3)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/3)*2)]['Time'])
                    elif dif >= 16*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/4)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/4)*2)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/4)*3)]['Time'])                       

            if df_shots_sh.iloc[i+1]['index 2min before'] - df_shots_sh.iloc[i]['index 2min after'] > 8*60*25:
                dif = df_shots_sh.iloc[i+1]['index 2min before'] - df_shots_sh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int(dif/2)]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int((dif/3)*2)]['Time'])
                elif dif >= 16*60*25:
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int(dif/4)]['Time'])
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*2)]['Time'])
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*3)]['Time'])
        except:
            if sh.iloc[-1].name - df_shots_sh.iloc[i]['index 2min after'] > 8*60*25:
                dif = sh.iloc[-1].name - df_shots_sh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int(dif/2)]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int((dif/3)*2)]['Time'])
                elif dif >= 16*60*25:
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int(dif/4)]['Time'])
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*2)]['Time'])
                    minutes_second_half.append(sh.loc[df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*3)]['Time'])
                    
    mean_pc_b_home = []
    mean_wpc_b_home = []
    mean_pc_a_home = []
    mean_wpc_a_home = []
    mean_pc_b_away = []
    mean_wpc_b_away = []
    mean_pc_a_away = []
    mean_wpc_a_away = []
    index_min_above = []
    index_min_below = []
    minutes = []
    seconds = []
    half = []

    #for each of the new time windows: add the pitch control and weighted pitch control values
    for i in range(len(minutes_first_half)):
        minute = float(int(minutes_first_half[i]))
        second = float((minutes_first_half[i] - int(minutes_first_half[i]))*60)
        index_above = fh[fh['Time'] > float(minute + second/60)].iloc[0].name
        index_below = fh[fh['Time'] <= float(minute + second/60)].iloc[-1].name
        index_2min_below = fh.index[fh.index >= index_below -2*60*25].min()
        index_1min_below = fh.index[fh.index >= index_below -60*25].min()
        index_2min_above = fh.index[fh.index <= index_above +2*60*25].max()
        mean_pc_b_home.append((fh.loc[index_below]['PC home previous minute'] + fh.loc[index_1min_below]['PC home previous minute'])/2)
        mean_wpc_b_home.append((fh.loc[index_below]['Weighted PC home previous minute'] + fh.loc[index_1min_below]['Weighted PC home previous minute'])/2)
        mean_pc_a_home.append((fh.loc[index_2min_above]['PC home previous minute'] + fh.loc[index_2min_above-60*25]['PC home previous minute'])/2)
        mean_wpc_a_home.append((fh.loc[index_2min_above]['Weighted PC home previous minute'] + fh.loc[index_2min_above-60*25]['Weighted PC home previous minute'])/2)
        mean_pc_b_away.append((fh.loc[index_below]['PC away previous minute'] + fh.loc[index_1min_below]['PC away previous minute'])/2)
        mean_wpc_b_away.append((fh.loc[index_below]['Weighted PC away previous minute'] + fh.loc[index_1min_below]['Weighted PC away previous minute'])/2)
        mean_pc_a_away.append((fh.loc[index_2min_above]['PC away previous minute'] + fh.loc[index_2min_above-60*25]['PC away previous minute'])/2)
        mean_wpc_a_away.append((fh.loc[index_2min_above]['Weighted PC away previous minute'] + fh.loc[index_2min_above-60*25]['Weighted PC away previous minute'])/2)
        index_min_below.append(index_2min_below)
        index_min_above.append(index_2min_above)
        minutes.append(minute)
        seconds.append(second)
        half.append(1)

    for i in range(len(minutes_second_half)):
        minute = float(int(minutes_second_half[i]))
        second = float((minutes_second_half[i] - int(minutes_second_half[i]))*60)
        index_above = sh[sh['Time'] > float(minute + second/60)].iloc[0].name
        index_below = sh[sh['Time'] <= float(minute + second/60)].iloc[-1].name
        index_2min_above = sh.index[sh.index <= index_above +2*60*25].max()
        index_2min_below = sh.index[sh.index >= index_below -2*60*25].min()
        index_1min_below = sh.index[sh.index >= index_below -60*25].min()
        mean_pc_b_home.append((sh.loc[index_below]['PC home previous minute'] + sh.loc[index_1min_below]['PC home previous minute'])/2)
        mean_wpc_b_home.append((sh.loc[index_below]['Weighted PC home previous minute'] + sh.loc[index_1min_below]['Weighted PC home previous minute'])/2)
        mean_pc_a_home.append((sh.loc[index_2min_above]['PC home previous minute'] + sh.loc[index_2min_above-60*25]['PC home previous minute'])/2)
        mean_wpc_a_home.append((sh.loc[index_2min_above]['Weighted PC home previous minute'] + sh.loc[index_2min_above-60*25]['Weighted PC home previous minute'])/2)
        mean_pc_b_away.append((sh.loc[index_below]['PC away previous minute'] + sh.loc[index_1min_below]['PC away previous minute'])/2)
        mean_wpc_b_away.append((sh.loc[index_below]['Weighted PC away previous minute'] + sh.loc[index_1min_below]['Weighted PC away previous minute'])/2)
        mean_pc_a_away.append((sh.loc[index_2min_above]['PC away previous minute'] + sh.loc[index_2min_above-60*25]['PC away previous minute'])/2)
        mean_wpc_a_away.append((sh.loc[index_2min_above]['Weighted PC away previous minute'] + sh.loc[index_2min_above-60*25]['Weighted PC away previous minute'])/2)
        index_min_below.append(index_2min_below)
        index_min_above.append(index_2min_above)
        minutes.append(minute + 45)
        seconds.append(second)
        half.append(2)

    index = []
    type_id = []
    type_name = []
    team_id = []
    frame_id = []
    team = []
    outcome = []
    match_id = []
    key_moment = []
    type_key = []

    #also add the other data in the original dataframe for the new time windows
    for i in range(len(half)):
        index.append(df_shots.iloc[-1].name +i)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        outcome.append(0)
        match_id.append(df_shots.iloc[0]['match_id'])
        key_moment.append('neutral')
        type_key.append(0)
        frame_id.append(0)

    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minutes,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
           'outcome': outcome,
           'match_id': match_id,
           'key_moment': key_moment,
            'type_key': type_key,
           'PC home 2min before': mean_pc_b_home,
            'WPC home 2min before': mean_wpc_b_home, 
            'PC home 2min after': mean_pc_a_home,
           'WPC home 2min after': mean_wpc_a_home,
           'PC away 2min before': mean_pc_b_away,
            'WPC away 2min before': mean_wpc_b_away, 
            'PC away 2min after': mean_pc_a_away,
           'WPC away 2min after': mean_wpc_a_away,
            'index 2min before': index_min_below,
            'index 2min after': index_min_above}

    #create a dataframe of the new time windows
    df = pd.DataFrame(data)
    #add this dataframe to the original shots and goals dataframe for each match
    df_new = pd.concat([df_shots, df])
    #create a complete dataframe for all the matches
    final_df = pd.concat([final_df, df_new])

#export the complete dataframe to a pickle file that can be used for regression
pickle.dump(final_df, open('input PC regression key moments on Momentum.p', 'wb'))  


In [ ]:
#two-minute time windows xT
from progressbar import ProgressBar
import pickle
import pandas as pd

#import the shots and goals dataframe
infile = open('shots and goals event dataframes.p','rb')
df_shot = pickle.load(infile)
infile.close()

#import the xT dataframe for all mathes
infile = open('all matches sliding windows xT.p','rb')
df_xT = pickle.load(infile)
infile.close()

#create an array with all the different matches in the xT dataframe
matches = df_xT.Match.unique()
pbar = ProgressBar()
#create empty dataframe
final_df = pd.DataFrame()

#iterate over each match in the array
for game in pbar(matches):
    
    #filter the shots and goals dataframe and the xt dataframe on each match
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    df_match_xT = df_xT.loc[df_xT['Match'] == game]
    #create separate dataframes for the first and second half
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    minutes_first_half = []
    
    #for each shot and goal in the dataframe: add whether it is favorable or unfavorable for the home team
    type_key = []
    key_mom = []
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['team'] == 'Home':
            key_mom.append('favorable')
            type_key.append(1)
        else:
            key_mom.append('unfavorable')
            type_key.append(-1)
    
    df_shots['key_moment'] = key_mom
    df_shots['type_key'] = type_key
    
    #check whether there are time windows in which no shot or gaol takes place. If yes, add the minutes to a list
    for i in range(len(df_shots_fh)):
        try:
            if i == 0:
                if int(df_shots_fh.iloc[i]['minute']) + int(df_shots_fh.iloc[i]['seconds'])/60  > 8:
                    dif = int(df_shots_fh.iloc[i]['minute']) + int(df_shots_fh.iloc[i]['seconds'])/60
                    if dif < 12:
                        minutes_first_half.append(dif/2)
                    elif dif >= 12 and dif <16:
                        minutes_first_half.append(dif/3)
                        minutes_first_half.append(2*dif/3)
                    elif dif >= 16:
                        minutes_first_half.append(1*dif/4)
                        minutes_first_half.append(2*dif/4)
                        minutes_first_half.append(3*dif/4)

            if (int(df_shots_fh.iloc[i+1]['minute'])+int(df_shots_fh.iloc[i+1]['seconds'])/60) - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60) > 8:                
                dif = (int(df_shots_fh.iloc[i+1]['minute'])+int(df_shots_fh.iloc[i+1]['seconds'])/60) - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 3*dif/4)

        except:
            if 46 - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60) > 8:
                dif = 46 - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 1*dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 3*dif/4)

    #do the same for the second half
    minutes_second_half = []
    for i in range(len(df_shots_sh)):
        try:
            if i == 0:
                if (int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60) - 45  > 8:
                    dif = (int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60) - 45
                    if dif < 12:
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - dif/2)
                    elif dif >= 12 and dif <16:
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - dif/3)
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 2*dif/3)
                    elif dif >= 16:
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 1*dif/4)
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 2*dif/4)
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 3*dif/4)

            if (int(df_shots_sh.iloc[i+1]['minute'])+int(df_shots_sh.iloc[i+1]['seconds'])/60) - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60) > 8:
                dif = (int(df_shots_sh.iloc[i+1]['minute'])+int(df_shots_sh.iloc[i+1]['seconds'])/60) - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 3*dif/4)

        except:
            if 91 - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60) > 8:
                dif = 91 - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 1*dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 3*dif/4)
        
    df_shots = df_shots[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'outcome', 'match_id', 'key_moment', 'type_key']]

    index = []
    type_id = []
    type_name = []
    team_id = []
    frame_id = []
    team = []
    outcome = []
    match_id = []
    key_moment = []
    type_key = []
    half = []
    minute = []
    seconds = []

    #for the new time windows, add the other data 
    for j in range(len(minutes_first_half)):
        index.append(df_shots_fh.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        outcome.append(0)
        match_id.append(df_shots_fh.iloc[0]['match_id'])
        key_moment.append('neutral')
        type_key.append(0)
        frame_id.append(0)
        half.append('1')
        minute.append(int(minutes_first_half[j]))
        seconds.append((minutes_first_half[j] - int(minutes_first_half[j]))*60)

    for j in range(len(minutes_second_half)):
        index.append(df_shots_sh.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        outcome.append(0)
        match_id.append(df_shots_sh.iloc[0]['match_id'])
        key_moment.append('neutral')
        type_key.append(0)
        frame_id.append(0)
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)

    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
           'outcome': outcome,
           'match_id': match_id,
           'key_moment': key_moment,
            'type_key': type_key}

    #create a dataframe for the new time windows
    df = pd.DataFrame(data)
    #add this dataframe to the shots and goals dataframe of that match
    df_new = pd.concat([df_shots, df])
    #add this complete match dataframe to the dataframe for all matches
    #this dataframe is still without xT values
    final_df = pd.concat([final_df, df_new])
        
            


In [ ]:
pbar = ProgressBar()
#create empty dataframe
df_final = pd.DataFrame()
#iterate over all matches
for game in pbar(matches):
    #again split on first and second half
    df_shots = final_df.loc[final_df['match_id'] == game]
    df_match_xT = df_xT.loc[df_xT['Match'] == game]
    df_xT_fh = df_match_xT.loc[df_match_xT['Half'] == 1]
    df_xT_sh = df_match_xT.loc[df_match_xT['Half'] == 2]
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    xt_home_2minb = []
    xt_away_2minb = []
    xt_home_2mina = []
    xt_away_2mina = []
    #for each row in the shots and goals dataframe: add the xt values for the time windows
    for i in range(len(df_shots_fh)):
        time = int(df_shots_fh.iloc[i]['minute']) + int(df_shots_fh.iloc[i]['seconds'])/60
        xt_home_2minb.append(df_xT_fh.loc[df_xT_fh['Minute'] <= time].iloc[-1]['xT_prev_2min_home'])
        xt_away_2minb.append(df_xT_fh.loc[df_xT_fh['Minute'] <= time].iloc[-1]['xT_prev_2min_away'])
        xt_home_2mina.append(df_xT_fh.loc[df_xT_fh['Minute'] <= time + 2].iloc[-1]['xT_prev_2min_home'])
        xt_away_2mina.append(df_xT_fh.loc[df_xT_fh['Minute'] <= time + 2].iloc[-1]['xT_prev_2min_away'])
    for i in range(len(df_shots_sh)):
        time = int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60
        xt_home_2minb.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_2min_home'])
        xt_away_2minb.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_2min_away'])
        xt_home_2mina.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time + 2].iloc[-1]['xT_prev_2min_home'])
        xt_away_2mina.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time + 2].iloc[-1]['xT_prev_2min_away'])
    
    df_shots['xT_prev_2min_home'] = xt_home_2minb
    df_shots['xT_next_2min_home'] = xt_home_2mina
    df_shots['xT_prev_2min_away'] = xt_away_2minb
    df_shots['xT_next_2min_away'] = xt_away_2mina
    
    #create a complete dataframe for all matches with the xT values
    df_final = pd.concat([df_shots, df_final])

#export the dataframe for all the matches to a pickle file that can be used for regression
pickle.dump(df_final, open('input xT regression key moments on Momentum.p', 'wb'))    